In [56]:
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from typing import TypedDict , Annotated
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages

In [57]:
load_dotenv(dotenv_path = "sa.env")

True

In [58]:
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.2", 
    task="text-generation",
    max_new_tokens=100,
    do_sample=True,
    temperature=0.0,
    streaming = True
)
model = ChatHuggingFace(llm=llm)

In [59]:
class qastate(TypedDict):
    messages : Annotated[list[BaseMessage], add_messages]
    state : str 

In [60]:
def analyze(state : qastate):
    response = state["messages"][-1].content
    prompt = f"Analyze the given content {response} \n Answer only in 4 words, If you find the content clear respond with :'answer', \n If you find the content Ambiguous respond with : 'clarify', \n If you find the content too broad respond with :'narrow', \n If you find the content that it is asking for the opinion respond with : 'opinion"
    st = model.invoke(prompt).content.lower().strip()

    if "answer" in st:
        st = "answer"
    elif 'clarify' in st:
        st = "clarify"
    elif "narrow" in st:
        st = "narrow"
    elif "opinion" in st:
        st = "opinion"
    else:
        st = "answer"  #Be safe

    return {"state" : st}


def answer(state : qastate):
    
    response = state["messages"]
    result = model.invoke(response).content

    return {"messages" : [AIMessage(content = result)]}  

def ambiguous(state : qastate):
    response = state["messages"]
    prompt = f"You find the content {response} ambiguous, Ask the user for the clarity in the content"
    am = model.invoke(prompt).content

    return {"messages" : [AIMessage(content = am)]}

def broad(state : qastate):
    response = state["messages"]
    prompt = f"You find the content {response} broad, Ask the user to narrow down the content."
    br = model.invoke(prompt).content

    return {"messages" : [AIMessage(content = br)]}

def opinion(state : qastate):
    response = state["messages"]
    prompt = f"You find the content {response} as opinion base, so you share your opinion as per your knowledge."
    pr = model.invoke(prompt).content

    return {"messages" : [AIMessage(content = pr)]}

def condition(state : qastate):
    if state["state"] == "answer":
        return "answer"
    elif state["state"] == "clarify":
        return "ambiguous"
    elif state["state"] == "narrow":
        return "broad"
    elif state["state"] == "opinion":
        return "opinion"



In [61]:
graph = StateGraph(qastate)

graph.add_node("analyze", analyze)
graph.add_node("answer", answer)
graph.add_node("ambiguous", ambiguous)
graph.add_node("broad", broad)
graph.add_node("opinion", opinion)

graph.add_edge(START, "analyze")
graph.add_conditional_edges("analyze",condition)
graph.add_edge("answer", END)
graph.add_edge("ambiguous", END)
graph.add_edge("broad", END)
graph.add_edge("opinion", END)


workflow = graph.compile()

In [62]:
result = workflow.invoke({"messages" : [HumanMessage(content = "Tell me about Tesla?")]})

ai_response  = result["messages"][-1].content

print(ai_response)

Could you please clarify what aspect of Tesla you're interested in? Are you asking about their current electric vehicle lineup, their automotive technology, their history and mission, or something else related to the company?
